In [23]:
import numpy as np
from scipy.stats import unitary_group
from collections import defaultdict

import matplotlib.pyplot as plt
np.set_printoptions(precision=3, suppress=True, linewidth=150) #print less digits of floats for readability
%matplotlib inline
%config InlineBackend.figure_format='retina' # makes the plots look nicer

In [24]:
def M(n, sigma, delta, m):
    r"""
    The Bell M matrix from Eq 1 of the paper
    n here represents the starting mode of MZI
    """
    mat = np.identity(m, dtype=np.complex128)
    mat[n, n] = np.exp(1j * sigma) * np.sin(delta)
    mat[n, n+1] = np.exp(1j * sigma) * np.cos(delta)
    mat[n+1, n] = np.exp(1j * sigma) * np.cos(delta)
    mat[n+1, n+1] = -np.exp(1j * sigma) * np.sin(delta)
    return mat

def P(j, phi, m):
    mat = np.identity(m, dtype=np.complex128)
    mat[j,j] = np.exp(1j * phi)
    return mat
    
def is_unitary(U):
    return np.allclose(U @ U.conj().T, np.eye(U.shape[0]))

In [32]:
def clements_decompose(U):
    V = U.conj()
    
    m = U.shape[0]
        
    phases = dict()
    phases['m'] = m
    phases['phi_ins'] = dict() # mode : phi
    phases['deltas'] = dict() # (mode, layer) : delta
    phases['sigmas'] = dict() # (mode, layer) : sigma
    phases['zetas'] = dict() # mode : zeta
    phases['phi_outs'] = dict() # mode : phi
        
    for j in range(m-1):
        #odd case in paper, because we index from 0 not 1
        if j % 2 == 0:
            x = m - 1
            y = j
            phi_j = np.angle(V[x, y+1]) - np.angle(V[x, y]) # reversed order from paper
            V = V @ P(j, phi_j, m)
            phases['phi_ins'][j] = phi_j
            for k in range(j+1):
                if V[x,y] == 0:
                    delta = 0.5 * np.pi
                else:
                    delta = np.arctan(-V[x,y+1] / V[x,y]).real # flipped from paper
                n = j - k
                V_temp = V @ M(n, 0, delta, m)
                sigma = np.angle(V_temp[x-1, y-1]) - np.angle(V_temp[x-1,y])
                V = V @ M(n, sigma, delta, m)
                phases['deltas'][n,k] = delta
                phases['sigmas'][n,k] = sigma
                x -= 1
                y -= 1
        else:
            x = m - j - 1
            y = 0
            phi_j = np.angle(V[x-1,y]) - np.angle(V[x,y])
            V = P(x, phi_j, m) @ V
            phases['phi_outs'][x] = phi_j
            for k in range(j+1):
                if V[x,y] == 0.:
                    delta = 0.5 * np.pi 
                else:
                    delta = np.arctan(V[x-1,y] / V[x,y]).real # flipped from paper
                V_temp = M(x-1, 0, delta, m) @ V
                n = m + k - j - 2
                if j != k:
                    sigma = (np.angle(V_temp[x+1, y+1]) - np.angle(V_temp[x,y+1]))
                else:
                    sigma = 0
                phases['deltas'][n,m-k-1] = delta
                phases['sigmas'][n,m-k-1] = sigma
                V = M(n, sigma, delta, m) @ V
                x += 1
                y += 1

    # these next two lines are just to remove a global phase
    zeta = - np.angle(V[0,0])
    V = V @ P(0, zeta, m)
    phases['zetas'][0] = zeta

    for j in range(1,m):
        zeta = np.angle(V[0,0]) - np.angle(V[j,j])
        V = V @ P(j, zeta, m)
        phases['zetas'][j] = zeta
        
    if not np.allclose(V, np.eye(m)):#is_unitary(V):
        raise Exception('decomposition failed')
    return phases

In [33]:
def clements_recompose(phases):
    m = phases['m']
    U = np.eye(m, dtype=np.complex128)
    
    #upper left of interferometer
    for j in range(0,m-1,2):
        phi_j = phases['phi_ins'][j]
        U = P(j, phi_j, m) @ U
        for k in range(j+1):
            n = j - k
            delta = phases['deltas'][n,k]
            sigma = phases['sigmas'][n,k]
            U = M(n, sigma, delta, m) @ U
            
    #diagonal phases
    for j in range(m):
        zeta = phases['zetas'][j]
        U = P(j, zeta, m) @ U
    
    #lower right of interferometer
    for j in reversed(range(1,m-1,2)):
        for k in reversed(range(j+1)):
            n = m + k - j - 2
            delta = phases['deltas'][n,m-k-1]
            sigma = phases['sigmas'][n,m-k-1]
            U = M(n, sigma, delta, m) @ U
            
    for j in range(1,m-1,2):
        x = m - j - 1
        phi_j = phases['phi_outs'][x]
        U = P(x, phi_j, m) @ U
    return U

In [34]:
def absorb_zeta(phases):
    m = phases['m']
    new_phases = phases.copy()
    del new_phases['zetas']
    new_phases['phi_edges'] = defaultdict(float) # (mode, layer) : phi
    
    if m % 2 == 0:
        new_phases['phi_outs'][0] = phases['zetas'][0]
        for j in range(1,m):
            zeta = phases['zetas'][j]
            layer = m - j 
            for mode in range(j,m-1,2):
                new_phases['sigmas'][mode, layer] += zeta
            for mode in range(j+1,m-1,2):
                new_phases['sigmas'][mode, layer-1] -= zeta
            if layer % 2 == 1:
                new_phases['phi_edges'][m-1, layer] += zeta
            else:
                new_phases['phi_edges'][m-1, layer-1] -= zeta
    else:
        for j in range(m):
            zeta = phases['zetas'][j]
            layer =  m - j - 1
            for mode in range(j,m-1,2):
                new_phases['sigmas'][mode, layer] += zeta
            for mode in range(j+1,m-1,2):
                new_phases['sigmas'][mode, layer-1] -= zeta
            if layer % 2 == 0:
                new_phases['phi_edges'][m-1, layer] += zeta
            else:
                new_phases['phi_edges'][m-1, layer-1] -= zeta
    return new_phases        

In [35]:
def rectangle_compact_decompose(U):
    phases_temp = clements_decompose(U)
    return absorb_zeta(phases_temp)

def rectangle_compact_compose(phases):
    m = phases['m']
    U = np.eye(m, dtype=np.complex128)
    for j in range(0,m-1,2):
        phi = phases['phi_ins'][j]
        U = P(j, phi, m) @ U
    for layer in range(m):
        phi_bottom = phases['phi_edges'][m-1, layer]
        if (layer + m + 1) % 2 == 0:
            U = P(m-1, phi_bottom, m) @ U
        for mode in range(layer % 2, m-1, 2):
            delta = phases['deltas'][mode, layer]
            sigma = phases['sigmas'][mode, layer]
            U = M(mode, sigma, delta, m) @ U 
    for j, phi_j in phases['phi_outs'].items():
        U = P(j, phi_j, m) @ U
    return U

In [42]:
m = 25
U = unitary_group.rvs(m)
# U = np.eye(m)
phases = rectangle_compact_decompose(U)
U_out = rectangle_compact_compose(phases)
np.allclose(U, U_out)

True

In [107]:
import numpy as np
from strawberryfields.program import Program
from strawberryfields.ops import Rgate,sMZgate
from strawberryfields.engine import Engine


m=4

prog = Program(m)

with prog.context as q:
    
    for i,phi in enumerate(params_list['P_']):
        Rgate(phi) | q[i]

    for j in range(m-1):
        for k in range(len(params_list[j])):
            delta = params_list[j][k][0]
            sigma = params_list[j][k][1]
            theta1 = delta + sigma
            theta2 = sigma - delta
            sMZgate(theta1, theta2) | (q[j-k], q[j-k+1])
 
    for i,phi in enumerate(params_list['Q_']):
        Rgate(phi) | q[m-i-1]
        
eng = Engine("fock", backend_options={"cutoff_dim": 10})
result = eng.run(prog)
state = result.state

KeyError: 2

# Clements

In [9]:
from strawberryfields.decomposition_tests import Clement_decomposition,Clement_reconstruction

In [19]:
m=6
U = unitary_group.rvs(m)

In [11]:
U

array([[ 0.063-0.407j,  0.493+0.128j,  0.047+0.115j, -0.258-0.599j,
         0.039+0.346j, -0.096+0.018j],
       [ 0.335+0.229j,  0.055-0.254j, -0.269-0.35j ,  0.114-0.062j,
         0.373+0.445j,  0.468-0.029j],
       [-0.046-0.162j, -0.333-0.331j,  0.68 -0.102j,  0.333-0.229j,
         0.162+0.166j, -0.126-0.213j],
       [-0.47 +0.163j, -0.036-0.458j, -0.458+0.036j,  0.2  -0.396j,
        -0.313+0.025j, -0.109-0.151j],
       [-0.053-0.031j, -0.105-0.061j,  0.14 +0.19j ,  0.051-0.385j,
         0.026-0.425j,  0.605+0.478j],
       [-0.577+0.231j,  0.113+0.463j,  0.052+0.218j,  0.216+0.002j,
         0.434+0.142j,  0.183-0.218j]])

In [12]:
params_list = Clement_decomposition(U)

V matrix is unitary: [[ 1.-0.j  0.+0.j -0.-0.j -0.-0.j  0.-0.j  0.+0.j]
 [-0.-0.j  1.-0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -0.-0.j  1.-0.j -0.+0.j -0.+0.j -0.+0.j]
 [-0.+0.j  0.-0.j -0.+0.j  1.-0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.-0.j  0.+0.j  1.-0.j  0.-0.j]
 [-0.-0.j -0.+0.j  0.+0.j  0.-0.j  0.-0.j  1.-0.j]]


In [13]:
def Clement_reconstruction(m,params_list):
    U_Q = np.identity(m, dtype=np.complex128)
    U_odd = np.identity(m, dtype=np.complex128)
    U_even = np.identity(m, dtype=np.complex128)

    for j in range(0,m-1,2):
        phi = params_list["P_"][j]
        U = P(j, phi, m) @ U
#         U_odd =  U_odd @ P(j,phi,m)
        for k in range(j+1):#range(len(params_list[j])):
            delta = params_list[j][k][0]
            sigma = params_list[j][k][1]
#             U_odd = U_odd@M(j-k, sigma, delta, m)
            U = M(j-k, sigma, delta, m) @ U
    for j in range(1,m-1,2):
        phi = params_list["P_"][j]
        U_even =    P(m-j-1, phi, m) @ U_even
        for k in range(len(params_list[j])):
            delta = params_list[j][k][0]
            sigma = params_list[j][k][1]
            U_even = M(m + k - j - 2, sigma, delta, m)@U_even
            

    for j in range(1,m):
        zeta = params_list["Q_"][j-1]
        U_Q = P(j, zeta, m) @ U_Q
        
    phase = P(0, params_list["global_zeta"][0], m)

    U_right = U_odd@phase@U_Q
    U_left = U_even
    return (U_right@U_left).T

array([[ 0.063-0.407j,  0.493+0.128j,  0.047+0.115j, -0.258-0.599j,
         0.039+0.346j, -0.096+0.018j],
       [ 0.335+0.229j,  0.055-0.254j, -0.269-0.35j ,  0.114-0.062j,
         0.373+0.445j,  0.468-0.029j],
       [-0.046-0.162j, -0.333-0.331j,  0.68 -0.102j,  0.333-0.229j,
         0.162+0.166j, -0.126-0.213j],
       [-0.47 +0.163j, -0.036-0.458j, -0.458+0.036j,  0.2  -0.396j,
        -0.313+0.025j, -0.109-0.151j],
       [-0.053-0.031j, -0.105-0.061j,  0.14 +0.19j ,  0.051-0.385j,
         0.026-0.425j,  0.605+0.478j],
       [-0.577+0.231j,  0.113+0.463j,  0.052+0.218j,  0.216+0.002j,
         0.434+0.142j,  0.183-0.218j]])

In [18]:
import numpy as np
from strawberryfields.program import Program
from strawberryfields.ops import Rgate,sMZgate, BSgate
from strawberryfields.engine import Engine


m=2

prog = Program(m)
eng = Engine('gaussian')

with prog.context as q:
    
    BSgate(0.25*np.pi,0) | (q[0], q[1])

result = eng.run(prog)

result.state.cov()

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [285]:
import sympy as s

In [286]:
theta1, theta2 = s.symbols(r'\theta_1 \theta_2', real=True)

In [287]:
BS = s.exp(-s.I * s.pi / 2) * s.Matrix([[1,s.I],[s.I,1]]) / s.sqrt(2)

In [288]:
middle = s.Matrix([[s.exp(s.I * (theta2+s.pi)), 0], [0, s.exp(s.I * (theta1))]])

In [290]:
sMZ = BS @ middle @ BS

In [292]:
s.simplify((sMZ @ sMZ.conjugate().T)) # check unitary

Matrix([
[1, 0],
[0, 1]])

In [293]:
sigma, delta = s.symbols(r'\Sigma \delta', real=True)

In [294]:
theta1_new = sigma + delta
theta2_new = sigma - delta

In [295]:
sMZp = sMZ.subs(((theta1, theta1_new), (theta2, theta2_new)))

In [296]:
sMZp.simplify()
sMZp

Matrix([
[   exp(I*(\Sigma + \delta))/2 - exp(I*(\Sigma - \delta + pi))/2, -I*(exp(I*(\Sigma + \delta)) + exp(I*(\Sigma - \delta + pi)))/2],
[-I*(exp(I*(\Sigma + \delta)) + exp(I*(\Sigma - \delta + pi)))/2,   -exp(I*(\Sigma + \delta))/2 + exp(I*(\Sigma - \delta + pi))/2]])

In [297]:
s.simplify(sMZp / (s.exp(s.I * sigma)))

Matrix([
[cos(\delta),  sin(\delta)],
[sin(\delta), -cos(\delta)]])